# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('../data_files/sp_500_stocks.csv')
from secrets_api import IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [7]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 67328178,
 'calculationPrice': 'close',
 'change': -6.64,
 'changePercent': -0.0395,
 'close': 168.07,
 'closeSource': 'iflcfoia',
 'closeTime': 1716194421963,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 169.32,
 'delayedPriceTime': 1708893517398,
 'extendedChange': -1.16,
 'extendedChangePercent': -0.00717,
 'extendedPrice': 163.9,
 'extendedPriceTime': 1734250918734,
 'high': 178.23,
 'highSource': 'urip1 mai5deleect  nyde',
 'highTime': 1729466198592,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 165.61,
 'iexCloseTime': 1732067945723,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 175.49,
 'iexOpenTime': 1714874334187,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1718440801310,
 'latestPrice': 167.16,
 'latestSource': 'Close',
 'latestTime': 'August 26, 2022',
 'latestUpdate': 1666186427606,
 'latestVolume': 79863196,
 'low': 1

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [10]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

27.43

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [11]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [19]:
df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    api_batch_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_batch_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data.keys():
            df = df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ], index=my_columns),
            ignore_index=True)

df

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/1969563

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,129.99,32.16,N/A
1,AAL,14.10,-4.75,N/A
2,AAP,181.59,20.22,N/A
3,AAPL,166.25,27.89,N/A
4,ABBV,138.65,20.08,N/A
...,...,...,...,...
496,YUM,115.10,22.41,N/A
497,ZBH,108.26,104.19,N/A
498,ZBRA,321.67,33.65,N/A
499,ZION,57.24,6.36,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [24]:
df.sort_values('Price-to-Earnings Ratio', inplace=True)
df = df[df['Price-to-Earnings Ratio'] > 0]
df = df[:50]
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,42.28,2.97,N/A
1,AIG,54.00,3.64,N/A
2,NUE,141.61,4.48,N/A
3,KSS,29.83,4.7,N/A
4,PHM,41.74,4.79,N/A
5,DHI,74.97,4.8,N/A
6,APA,40.98,4.8,N/A
7,PVH,65.28,4.81,N/A
8,PFG,76.81,4.84,N/A
9,MGM,34.79,4.92,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [26]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [31]:
portfolio_input()

Enter the value of your portfolio:1,000,000
That's not a number! 
 Try again:
Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [32]:
portfolio_size

'1000000'

In [34]:
position_size = float(portfolio_size)/len(df.index)
for row in df.index:
    df.loc[row, 'Number of Shares to Buy'] = position_size/df.loc[row,'Price']

df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,42.28,2.97,473.036897
1,AIG,54.00,3.64,370.37037
2,NUE,141.61,4.48,141.232964
3,KSS,29.83,4.7,670.465974
4,PHM,41.74,4.79,479.156684
5,DHI,74.97,4.8,266.773376
6,APA,40.98,4.8,488.042948
7,PVH,65.28,4.81,306.372549
8,PFG,76.81,4.84,260.382763
9,MGM,34.79,4.92,574.877838


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [40]:
symbol = 'AAPL'

batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#print(data['AAPL']['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = ev/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gp = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = ev/gp


16.45401361179606


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [79]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earn Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        #print(data[symbol]['quote'])
        if symbol in data.keys():
            ev = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gp = data[symbol]['advanced-stats']['grossProfit']

            try:
                ev_to_ebitda = ev/ebitda
            except TypeError:
                ev_to_ebitda = np.NaN
            
            try:
                ev_to_gp = ev/gp
            except TypeError:
                ev_to_gp = np.NaN
                
            rv_df = rv_df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gp,
                'N/A',
                'N/A'
            ], index=rv_columns),
            ignore_index=True)

rv_df

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_50416/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,135.79,N/A,31.67,N/A,7.85,N/A,6.23,N/A,22.299311,N/A,11.397665,N/A,N/A
1,AAL,13.96,N/A,-4.94,N/A,-1.1,N/A,0.2298,N/A,109.621740,N/A,0.845300,N/A,N/A
2,AAP,175.55,N/A,20.52,N/A,3.87,N/A,0.9843,N/A,11.026857,N/A,2.288019,N/A,N/A
3,AAPL,169.12,N/A,28.17,N/A,46.19,N/A,6.92,N/A,21.352105,N/A,16.364578,N/A,N/A
4,ABBV,138.05,N/A,19.49,N/A,17.19,N/A,4.2,N/A,10.687080,N/A,7.471790,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.69,N/A,22.64,N/A,-3.75,N/A,4.95,N/A,19.238439,N/A,8.956288,N/A,N/A
497,ZBH,111.28,N/A,104.5,N/A,1.91,N/A,3.06,N/A,17.762723,N/A,5.508005,N/A,N/A
498,ZBRA,316.80,N/A,33.64,N/A,6.63,N/A,2.81,N/A,16.734521,N/A,6.762079,N/A,N/A
499,ZION,58.13,N/A,6.64,N/A,1.65,N/A,2.49,N/A,5.205814,N/A,2.509642,N/A,N/A


In [ ]:
rv_df

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [80]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,184.700,N/A,61.24,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,299.280,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,296.120,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,97.980,N/A,48.84,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.500,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,51.120,N/A,14.84,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,59.780,N/A,32.29,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,32.380,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,105.150,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [81]:
for col in ['Price-to-Earn Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_df[col].fillna(rv_df[col].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [82]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [133]:
metrics = {
    'Price-to-Earn Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore(rv_df[metric], rv_df.loc[row, metric]) / 100

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,31.15,642.054575,4.48,0.14,0.8805,0.24,0.2017,0.04,3.508576,0.2,0.999324,0.22,0.168
1,AIG,55.00,363.636364,3.53,0.1,0.9399,0.32,0.7282,0.56,2.010447,0.02,0.690027,0.08,0.216
2,PVH,64.18,311.623559,4.67,0.16,0.8151,0.14,0.4736,0.26,4.791551,0.46,1.112450,0.32,0.268
3,GPS,10.18,1964.636542,-52.19,0.02,1.5000,0.73,0.2314,0.06,5.227787,0.56,0.760590,0.1,0.294
4,CAH,69.60,287.356322,-20.65,0.04,-26.7400,0.02,0.1054,0.02,6.910501,0.78,2.835935,0.7,0.312
5,UNM,39.85,501.882058,7.42,0.54,0.8291,0.2,0.6577,0.48,4.003083,0.3,0.648513,0.06,0.316
6,GM,40.89,489.117144,7.60,0.59,0.8947,0.26,0.4430,0.24,2.347465,0.04,1.783568,0.46,0.318
7,NRG,41.78,478.697942,2.98,0.08,1.9200,0.88,0.3438,0.16,2.839991,0.06,2.336554,0.6,0.356
8,COF,113.22,176.647235,5.00,0.26,0.8212,0.18,1.2800,0.9,3.306917,0.14,1.282119,0.38,0.372
9,SYF,35.21,568.020449,5.00,0.26,1.2900,0.58,0.9381,0.72,3.087896,0.08,1.009649,0.24,0.376


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [134]:
from statistics import mean

for row in rv_df.index:
    value_percentile = []
    for metric in metrics.keys():
        value_percentile.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentile)

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,31.15,642.054575,4.48,0.14,0.8805,0.24,0.2017,0.04,3.508576,0.2,0.999324,0.22,0.168
1,AIG,55.00,363.636364,3.53,0.1,0.9399,0.32,0.7282,0.56,2.010447,0.02,0.690027,0.08,0.216
2,PVH,64.18,311.623559,4.67,0.16,0.8151,0.14,0.4736,0.26,4.791551,0.46,1.112450,0.32,0.268
3,GPS,10.18,1964.636542,-52.19,0.02,1.5000,0.73,0.2314,0.06,5.227787,0.56,0.760590,0.1,0.294
4,CAH,69.60,287.356322,-20.65,0.04,-26.7400,0.02,0.1054,0.02,6.910501,0.78,2.835935,0.7,0.312
5,UNM,39.85,501.882058,7.42,0.54,0.8291,0.2,0.6577,0.48,4.003083,0.3,0.648513,0.06,0.316
6,GM,40.89,489.117144,7.60,0.59,0.8947,0.26,0.4430,0.24,2.347465,0.04,1.783568,0.46,0.318
7,NRG,41.78,478.697942,2.98,0.08,1.9200,0.88,0.3438,0.16,2.839991,0.06,2.336554,0.6,0.356
8,COF,113.22,176.647235,5.00,0.26,0.8212,0.18,1.2800,0.9,3.306917,0.14,1.282119,0.38,0.372
9,SYF,35.21,568.020449,5.00,0.26,1.2900,0.58,0.9381,0.72,3.087896,0.08,1.009649,0.24,0.376


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [135]:
rv_df = rv_df.sort_values('RV Score')[:50]
rv_df.reset_index(drop=True, inplace=True)
#rv_df.drop(columns=['index'], inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,31.15,642.054575,4.48,0.14,0.8805,0.24,0.2017,0.04,3.508576,0.2,0.999324,0.22,0.168
1,AIG,55.00,363.636364,3.53,0.1,0.9399,0.32,0.7282,0.56,2.010447,0.02,0.690027,0.08,0.216
2,PVH,64.18,311.623559,4.67,0.16,0.8151,0.14,0.4736,0.26,4.791551,0.46,1.112450,0.32,0.268
3,GPS,10.18,1964.636542,-52.19,0.02,1.5000,0.73,0.2314,0.06,5.227787,0.56,0.760590,0.1,0.294
4,CAH,69.60,287.356322,-20.65,0.04,-26.7400,0.02,0.1054,0.02,6.910501,0.78,2.835935,0.7,0.312
5,UNM,39.85,501.882058,7.42,0.54,0.8291,0.2,0.6577,0.48,4.003083,0.3,0.648513,0.06,0.316
6,GM,40.89,489.117144,7.60,0.59,0.8947,0.26,0.4430,0.24,2.347465,0.04,1.783568,0.46,0.318
7,NRG,41.78,478.697942,2.98,0.08,1.9200,0.88,0.3438,0.16,2.839991,0.06,2.336554,0.6,0.356
8,COF,113.22,176.647235,5.00,0.26,0.8212,0.18,1.2800,0.9,3.306917,0.14,1.282119,0.38,0.372
9,SYF,35.21,568.020449,5.00,0.26,1.2900,0.58,0.9381,0.72,3.087896,0.08,1.009649,0.24,0.376


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [137]:
portfolio_input()

Enter the value of your portfolio:1000000


In [138]:
position_size = float(portfolio_size)/len(rv_df.index)

for row in rv_df.index:
    rv_df.loc[row, 'Number of Shares to Buy'] = position_size/rv_df.loc[row, 'Price']

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,31.15,642.054575,4.48,0.14,0.8805,0.24,0.2017,0.04,3.508576,0.2,0.999324,0.22,0.168
1,AIG,55.00,363.636364,3.53,0.1,0.9399,0.32,0.7282,0.56,2.010447,0.02,0.690027,0.08,0.216
2,PVH,64.18,311.623559,4.67,0.16,0.8151,0.14,0.4736,0.26,4.791551,0.46,1.112450,0.32,0.268
3,GPS,10.18,1964.636542,-52.19,0.02,1.5000,0.73,0.2314,0.06,5.227787,0.56,0.760590,0.1,0.294
4,CAH,69.60,287.356322,-20.65,0.04,-26.7400,0.02,0.1054,0.02,6.910501,0.78,2.835935,0.7,0.312
5,UNM,39.85,501.882058,7.42,0.54,0.8291,0.2,0.6577,0.48,4.003083,0.3,0.648513,0.06,0.316
6,GM,40.89,489.117144,7.60,0.59,0.8947,0.26,0.4430,0.24,2.347465,0.04,1.783568,0.46,0.318
7,NRG,41.78,478.697942,2.98,0.08,1.9200,0.88,0.3438,0.16,2.839991,0.06,2.336554,0.6,0.356
8,COF,113.22,176.647235,5.00,0.26,0.8212,0.18,1.2800,0.9,3.306917,0.14,1.282119,0.38,0.372
9,SYF,35.21,568.020449,5.00,0.26,1.2900,0.58,0.9381,0.72,3.087896,0.08,1.009649,0.24,0.376


In [139]:
# Checksum that we're spending the right amount of money!
amount = []
for row in rv_df.index:
    amount.append(rv_df.loc[row, 'Number of Shares to Buy'] * rv_df.loc[row, 'Price'])

sum(amount)

1000000.0

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [143]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [144]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [145]:
column_formats = {
                    'A':['Ticker', string_template],
                    'B':['Price', dollar_template],
                    'C':['Number of Shares to Buy', integer_template],
                    'D':['Price-to-Earn Ratio', float_template],
                    'E':['PE Percentile', percent_template],
                    'F':['Price-to-Book Ratio', float_template],
                    'G':['PB Percentile', percent_template],
                    'H':['Price-to-Sales Ratio', float_template],
                    'I':['PS Percentile', percent_template],
                    'J':['EV/EBITDA', float_template],
                    'K':['EV/EBITDA Percentile', percent_template],
                    'L':['EV/GP', percent_template],
                    'M':['EV/GP Percentile', percent_template],
                    'N':['RV Score', float_template]
                }

for col in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{col}:{col}', 25, column_formats[col][1])
    writer.sheets['Value Strategy'].write(f'{col}1', column_formats[col][0], column_formats[col][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [146]:
writer.save()